## Разделим текст на сообщения

In [8]:
import re

In [9]:
text = open('ling_emails.txt','r').read()
messages = re.findall('ЭТОМОЁПИСЬМОНОМЕР:[0-9]*(.+?)ЭТОКОНЕЦПИСЬМАНОМЕР', text, flags = re.DOTALL)

In [1017]:
pronouns = open('pronouns.txt','r').read()
slovar = open('zdf-win.txt','r').read()

## Уберем всякую ерунду из писем (цитаты, переадресовки и т.п.)

In [729]:
def cleaner(text):
    text1 = ''
    for l in range(len(text)): #убираем разметку жирного шрифта
        if text[l] != '*'or (text[l] == '*' and text[l-1] in '0123456789' and text[l+1] in '0123456789'):
                text1 += text[l]
        
    lines0 = text1.split('\n')
    n = 0
    lines = []
    for i in lines0:
        if i != '':
            lines.append(i)
    i = 0
    j = 0
    while i != len(lines):
        if len(re.findall('[0-9]*:[0-9]* пользователь', lines[i])) > 0 or len(re.findall('On Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday, [0-9]+ September|October|November|December|January|February|March|April|May|June|July|August [0-9]+, ', lines[i])) > 0 or len(re.findall('_____', lines[i])) > 0 or len(re.findall('Отправлено со', lines[i])) > 0 or len(re.findall('Sent from', lines[i]))>0 or len(re.findall('> ', lines[i])) > 0 or len(re.findall('---- Пересылаемое сообщение ----', lines[i]))>0 or len(re.findall('[0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}:[0-9]{2} GMT', lines[i]))>0 or len(re.findall('[0-9]{2}\.[0-9]{2}\.[0-9]{4}, [0-9]{2}:[0-9]{2}, "', lines[i]))>0 or len(re.findall('пользователь.*написал', lines[i]))>0:
            lines = lines[:i]
            lines_str = ''
            for j in lines:
                lines_str += j
                lines_str += ' '
            break
        if len(re.findall('Копия:', lines[i])) > 0:
            lines = lines[i+1:]
            lines_str = ''
            for j in lines:
                lines_str += j
                lines_str += ' '
            break
        i += 1
    lines_str = ''
    for j in lines:
        lines_str += j
        lines_str += ' '
    return lines_str

In [64]:
r = re.search('aa', 'nmlaa')
r.end()

5

In [34]:
def rev(x):
    y = ''
    for i in reversed(x):
        y += i
    return y

In [774]:
def sklonenie(x):
    d = re.search('[0-9]+([а-яА-Я]+)', x)
    if d is not None and d.group(1) in ['го', 'им', 'ым', 'му', 'ом', 'ем', 'ый', 'ий', 'й', 'я', 'е']:
        return True
    else:
        return False

In [749]:
def lstrip(x):
    punct = []
    while len(x) > 0 and x[0] in ['?', '!', '.', ')', '(', ';', ':', '"', '-', '—', '*' , ',', '«', '»', "№", '[', ']', '\u200b']:
        if x[0] == '.':
            punct.append(re.match('\.+', x).group(0)) #нашли все многоточия (в начале и в конце)
            x = x.lstrip('\.+') #убрали многоточие
        elif x[0] == '?' or x[0] == '!':
            punct.append(re.match('((!|\?)+)', x).group(0))
            x = x.lstrip('((!|\?)+)')
        elif x[0] == ')':
            punct.append(re.match('\)+-*:*', x).group(0)) #проверка на веселые смайлики
            x = x.lstrip(re.match('\)+-*:*', x).group(0))
        elif x[0] == '(':
            punct.append(re.match('\(+-*:*', x).group(0)) #проверка на грустные смайлики
            x = x.lstrip(re.match('\(+-*:*', x).group(0))
        elif x[0] == ':' and x[1] == '(':
            punct.append(re.match(':-*\(+', x).group(0))
            x = x.lstrip(re.match(':-*\(+', x).group(0))
        elif x[0] == ':' and x[1] == ')':
            punct.append(re.match(':-*\)+', x).group(0)) #проверка на веселые смайлики
            x = x.lstrip(re.match(':-*\)+', x).group(0))
        elif x[0] == '\u200b':
            x = x[1:]
        else:
            p = re.match(']|\)|;|:|"|-|—|\*|,|\(|«|»|№|\[', x).group(0)
            punct.append(p)
            x = x.lstrip(p)
    num = re.search('([0-9]+[а-яА-Яa-zA-Z]+\.{0,1})', x)
    if sklonenie(x) != True and num is not None and num.group(0) == x:
        num1 = re.search('([0-9]+)([а-яА-Яa-zA-Z]+\.{0,1})', x)
        x = num1.group(2)
        punct.append(num1.group(1))
    return x, punct

In [599]:
def rstrip(x):
    punct = []
    x1 = rev(x)
    while len(x1) > 0 and x1[0] in ['?', '!', '.', ')', '(', ';', ':', '"', '-', '—', '*' , ',', '«', '»', '[', ']', '%', '$']:
        if x1[0] == '.':
            if init(rev(x1)) != True and red(rev(x1)) != True and cifra(x1[1:]) != True:
                punct.append(re.match('\.+', x1).group(0)) #нашли все многоточия (в начале и в конце)
                x1 = x1.lstrip(re.match('\.+', x1).group(0)) #убрали многоточие
            else:
                break
        elif x1[0] == '?' or x1[0] == '!':
            punct.append(re.match('((!|\?)+)', x1).group(0))
            x1 = x1.lstrip(re.match('((!|\?)+)', x1).group(0))
            x = rev(x1)
        elif x1[0] == ')':
            punct.append(re.match('\)+-*:*', x1).group(0)) #проверка на веселые смайлики
            x1 = x1.lstrip(re.match('\)+-*:*', x1).group(0))
        elif x1[0] == '(':
            punct.append(re.match('\(+-*:*', x1).group(0)) #проверка на грустные смайлики
            x1 = x1.lstrip(re.match('\(+-*:*', x1).group(0))
        
        else:
            p = re.match(']|;|:|"|-|—|\*|,|«|»|\[|$|%', x1).group(0)
            punct.append(p)
            x1 = x1.lstrip(p)
            x = rev(x1)
    num = re.search('([0-9]+\.{0,1}[а-яА-Яa-zA-Z]+)', x1)
    if num is not None and num.group(0) == x1:
        num1 = re.search('([0-9]+)(\.{0,1}[а-яА-Яa-zA-Z]+)', x1)
        x1 = num1.group(2)
        punct.append(num1.group(1))
    for i in range(len(punct)): #перевернуть скобки обратно
        punct[i] = rev(punct[i])
    return rev(x1), list(reversed(punct))

In [1123]:
def red(x):
    reduces = ['д.', 'доб.', 'каб.', 'ул.', 'тел.', 'м.', 'пер.', 'руб.', 'e.g.', 'т.к.', 'гг.', 'г.', 'тыс.', 'тысч.', 'англ.', 'д.э.н.', 'к.ф.н.', 'д.ф.н.', 'Inc.', 'ауд.', 'ноябр.', 'дек.', 'янв.', 'апр.', 'сент.', 'окт.', 'фев.', 'п.', 'к.', 'т.е.', 'л.', 'к.и.н.', 'см.', 'прил.', 'стр.', 'в.н.с.', 'мин.', 'М.', 'т.п.', 'P.S.', 'м.б.', 'им.', 'к.', 'н.', 'филос.']
    red_occ = re.search('([А-Я]\.)', x)
    if x in reduces or x.lower() in reduces:
        return True
    elif red_occ is not None:
        if red_occ.group(0) == x:
            return True
    else:
        return False

In [570]:
def cifra(x):
    if x in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X']:
        return True
    else:
        return False

In [38]:
def init(x):
    flag = False
    s = re.search('([А-Я]\.[А-Я]\.)', x)
    if s is not None:
        if s.group(0) == x:
            flag = True
    return flag

In [661]:
def interval(x):
    s = re.search('([0-9]+-[0-9]+)', x)
    if s is not None and s.group(0) == x:
        return True
    else:
        return False

In [674]:
def house(x):
    s = re.search('[0-9]+/[0-9]+', x)
    if s is not None and s.group(0) == x:
        return True
    else:
        return False

In [1081]:
def streets(words):
    streets = open('double_street_name.txt','r').read()
    streets1 = streets.split('\n')
    a = []
    for i in streets1:
        a.append(i.split(' '))
    words1 = []
    i = 0
    bi = 'abracadabra'
    while i < len(words)-1:
        word = words[i]
        flag = 1
        bi = words[i] + ' ' + words[i+1]
        for j in a:
            if len(words[i]) > 3 and (len(words[i+1]) > 3) and (re.match(words[i][:-3], j[0]) is not None) and (re.match(words[i+1][:-3], j[1]) is not None):
                word = bi
                flag = 2
                break
        words1.append(word)
        i += flag
    words1.append(words[-1])
    return words1

In [1041]:
def phone(words):
    words1 = []
    i = 0
    while i < len(words)-2:
        word = words[i]
        print(words[i])
        flag = 1
        tri = words[i] + ' ' + words[i+1] + ' ' + words[i+2]
        if i < len(words)-3:
            quat = words[i] + words[i+1] + words[i+2] + ' ' + words[i+3]
            third = re.match('[0-9]+', words[i+1])
            forth = re.match('[0-9]{3}-', words[i+3])
        first = re.match('\+{0,1}[0-9]+', words[i])
        second = re.match('[0-9]{3}-', words[i+2])
        if i < len(words)-3 and words[i] == '('and third is not None and third.group(0) == words[i+1] and words[i+2] == ')' and forth is not None:
            word = quat
            flag = 4
        elif first is not None and first.group(0) == words[i] and words[i+1] == '-' and second is not None:
            word = tri
            flag = 3
        words1.append(word)
        i += flag
    if i < len(words)-1:
        words1 += words[i:]
    return words1

In [979]:
def add(words):
    words1 = []
    i = 0
    while i < len(words)-3:
        word = words[i]
        flag = 1
        quat = words[i] + ' ' + words[i+1] + ' ' + words[i+2] + ' ' + words[i+3]
        first = re.search('[0-9]+', words[i+1])
        second = re.search('[0-9]+', words[i+3])
        if words[i] == 'доб.' and first.group(0) == words[i+1] and words[i+2] == '-' and second.group(0) == words[i+3]:
            words1.append('доб.')
            word = words[i+1] + ' ' + words[i+2] + ' ' + words[i+3]
            flag = 4
        words1.append(word)
        i += flag
    if i < len(words)-1:
        words1 += words[i:]
    return words1

In [1030]:
rstrip('все-таки')

('все-таки', [])

In [1113]:
def splitter(text):
    if text == '':
        return ''
    
    if text in messages:
        text = cleaner(text)
    words = text.split() #по умолчанию делим по пробелу, единые сущности, разделенные пробелом, будут рассмотрены потом
    words2 = []
    
    for x in range(len(words)):
        lstr = lstrip(words[x]) #стрип слева
        if len(lstr[1]) != 0:
            words2 += lstr[1] #знаки препинания перед словом
        rstr = rstrip(lstr[0]) #стрип справа
        if '(' in rstr[0]: #проверка на скобку внутри слова, часто перед ней нет пробела
            words2.append(rstr[0].split('(')[0])
            words2.append('(')
            words2.append(rstr[0].split('(')[1])
        elif '/' in rstr[0]  and 'www' not in rstr[0] and 'http' not in rstr[0] and 'ru' not in rstr[0] and house(rstr[0]) != True: #проверка на слеш внутри слова
            words2.append(rstr[0].split(re.search('/+', rstr[0]).group(0))[0])
            words2.append(re.search('/+', rstr[0]).group(0))
            words2.append(rstr[0].split(re.search('/+', rstr[0]).group(0))[1])
        elif ',' in rstr[0]: #проверка на запятую внутри слова
            words2.append(rstr[0].split(',')[0])
            words2.append(',')
            words2.append(rstr[0].split(',')[1])
        elif '-' in rstr[0] and 'www' not in rstr[0] and 'http' not in rstr[0] and 'ru' not in rstr[0]:
            if interval(rstr[0]) == True or (rstr[0] not in pronouns and rstr[0] not in slovar):
                words2.append(rstr[0].split('-')[0])
                words2.append('-')
                words2.append(rstr[0].split('-')[1])
            else:
                words2.append(rstr[0])
        else:
            words2.append(rstr[0]) #итоговое слово
        if len(rstr[1]) != 0:
            words2 += rstr[1] #знаки препинания после слова
            
    words3 = []
    for i in words2:
        if i != '':
            words3.append(i)
    
#    words3 = streets(words3) #соединение единых токенов, разделенных пробелом (улицы, телефоны)         
    words3 = phone(words3)
    words3 = add(words3)

    if len(words3) > 0:
        if words3[-1] == '-':
            words3 = words3[:-1]
        
    return words3

# Тестирование токенизатора

Создадим тестовый текст длиной >500 токенов.

In [1104]:
l = 0
test = ''
for i in range(800, 806):
    l += len(splitter(messages[i]))
    test += cleaner(messages[i])
for i in range(810, 815):
    l += len(splitter(messages[i]))
    test += cleaner(messages[i])
print(l) #685 токенов

Здравствуйте
!
В
последний
момент
мы
обнаружили
несостыковку
с
расписанием
сессии
на
сайте
(
http://www.hse.ru/ba/ling/timetable?fromdate=2015.06.22&todate=2015.06.27&groupoid=399&timetable-courses=1&timetable-groups=399
)
,
где
экзамен
начинается
в
9
,
и
тем
,
что
было
вывешено
учебный
офисом
(
10
часов).Во
сколько
на
самом
деле
нужно
приходить
?
Вся
группа
считала
,
что
приходить
нужно
к
10
.
С
уважением
,
Давайте
в
10
.
Только
чтобы
все
пришли
к
10
и
чтобы
не
было
путаницы
.
Всего
доброго
,
Дорогие
студенты
,
Завтра
мы
ждем
вас
ВСЕХ
на
собрании
с
руководителем
школы
лингвистики
Е.В.
Рахилиной
.
11.06.2015
в
13:40
в
511
аудитории
.
Это
не
займет
больше
20
минут
,
при
этом
те
,
у
кого
в
это
время
пары
,
смогут
пойти
на
занятия
после
собрания
.
С
уважением
,
Здравствуйте
,
lingua
!
Пароль
Вашего
аккаунта
Google
(
linghse2014@gmail.com
)
был
изменен
.
Если
пароль
изменили
Вы
,
то
дальнейшие
действия
не
требуются
.
Если
Вы
не
меняли
пароль
,
возможно
,
Ваш
аккаунт
был
взломан
.
Чтобы
пол

In [1101]:
f = open('test.txt', 'w')

In [1102]:
f.write(test)

4136

In [1105]:
f.close()

In [1106]:
test = open('test.txt', 'r').read()

In [1126]:
test

'Здравствуйте! В последний момент мы обнаружили несостыковку с расписанием сессии на сайте (http://www.hse.ru/ba/ling/timetable?fromdate=2015.06.22&todate=2015.06.27&groupoid=399&timetable-courses=1&timetable-groups=399), где экзамен начинается в 9 , и тем, что было вывешено учебный офисом (10 часов).Во сколько на самом деле нужно приходить? Вся группа считала, что приходить нужно к 10. С уважением, группа 1401 Давайте в 10. Только чтобы все пришли к 10 и чтобы не было путаницы. Всего доброго, Борис Орехов Дорогие студенты, Завтра мы ждем вас ВСЕХ на собрании с руководителем школы лингвистики Е.В. Рахилиной. 11.06.2015 в 13:40 в 511 аудитории. Это не займет больше 20 минут, при этом те, у кого в это время пары, смогут пойти на занятия после собрания. С уважением, Учебный офис Здравствуйте, lingua! Пароль Вашего аккаунта Google (linghse2014@gmail.com) был изменен. Если пароль изменили Вы, то дальнейшие действия не требуются. Если Вы не   меняли пароль, возможно, Ваш аккаунт был взломан.

In [1125]:
splitter(test)

Здравствуйте
!
В
последний
момент
мы
обнаружили
несостыковку
с
расписанием
сессии
на
сайте
(
http://www.hse.ru/ba/ling/timetable?fromdate=2015.06.22&todate=2015.06.27&groupoid=399&timetable-courses=1&timetable-groups=399
)
,
где
экзамен
начинается
в
9
,
и
тем
,
что
было
вывешено
учебный
офисом
(
10
часов).Во
сколько
на
самом
деле
нужно
приходить
?
Вся
группа
считала
,
что
приходить
нужно
к
10
.
С
уважением
,
группа
1401
Давайте
в
10
.
Только
чтобы
все
пришли
к
10
и
чтобы
не
было
путаницы
.
Всего
доброго
,
Борис
Орехов
Дорогие
студенты
,
Завтра
мы
ждем
вас
ВСЕХ
на
собрании
с
руководителем
школы
лингвистики
Е.В.
Рахилиной
.
11.06.2015
в
13:40
в
511
аудитории
.
Это
не
займет
больше
20
минут
,
при
этом
те
,
у
кого
в
это
время
пары
,
смогут
пойти
на
занятия
после
собрания
.
С
уважением
,
Учебный
офис
Здравствуйте
,
lingua
!
Пароль
Вашего
аккаунта
Google
(
linghse2014@gmail.com
)
был
изменен
.
Если
пароль
изменили
Вы
,
то
дальнейшие
действия
не
требуются
.
Если
Вы
не
меняли
пароль
,
возможно

['Здравствуйте',
 '!',
 'В',
 'последний',
 'момент',
 'мы',
 'обнаружили',
 'несостыковку',
 'с',
 'расписанием',
 'сессии',
 'на',
 'сайте',
 '(',
 'http://www.hse.ru/ba/ling/timetable?fromdate=2015.06.22&todate=2015.06.27&groupoid=399&timetable-courses=1&timetable-groups=399',
 ')',
 ',',
 'где',
 'экзамен',
 'начинается',
 'в',
 '9',
 ',',
 'и',
 'тем',
 ',',
 'что',
 'было',
 'вывешено',
 'учебный',
 'офисом',
 '(',
 '10',
 'часов).Во',
 'сколько',
 'на',
 'самом',
 'деле',
 'нужно',
 'приходить',
 '?',
 'Вся',
 'группа',
 'считала',
 ',',
 'что',
 'приходить',
 'нужно',
 'к',
 '10',
 '.',
 'С',
 'уважением',
 ',',
 'группа',
 '1401',
 'Давайте',
 'в',
 '10',
 '.',
 'Только',
 'чтобы',
 'все',
 'пришли',
 'к',
 '10',
 'и',
 'чтобы',
 'не',
 'было',
 'путаницы',
 '.',
 'Всего',
 'доброго',
 ',',
 'Борис',
 'Орехов',
 'Дорогие',
 'студенты',
 ',',
 'Завтра',
 'мы',
 'ждем',
 'вас',
 'ВСЕХ',
 'на',
 'собрании',
 'с',
 'руководителем',
 'школы',
 'лингвистики',
 'Е.В.',
 'Рахилиной',


Ошибки:
    1. часов).Во 
    2. а 
        )
    3. в
        10
        -
        3-
    4. tkulakova@hse.ru<mailto:tkulakova@hse.ru>

### Точность: 0.99564
### Полнота: 0.99132

А теперь проверим на тексте, не имеющем отношения к письмам

In [1128]:
test2 = open('test-2.txt', 'r').read()

In [1129]:
test2

'Text segmentation in OpenCorpora project\nПод сегментацией текста в данной статье подразумевается деление текста на слова и предложения. В рамках OpenCorpora была проведена работа по сегментации текстов общим объёмом более 600\xa0тыс. словоупотреблений. На основе полученных данных была обучена математическая модель сегментации текста. Модель представляет собой вероятностный бинарный классификатор.\n\nПроблема с текстами\nВсе тексты, входящие в корпус, опубликованы на условиях, совместимых с лицензией СС-BY-SA. Это позволяет сделать корпус целиком доступным, но сужает круг материалов.\n\nВходные данные\nПри проектировании модуля сегментации важно понимать, что никакая лингвистическая информация по умолчанию не предполагается доступной во время сегментации.  На вход модуль сегментации получает последовательность знаков (буквы кириллицы и других алфавитов, знаков пунктуации, цифр, пробельных символов и других знаков, предусмотренных кодировкой знаков Unicode).\n\nЕдиницы сегментации и кл

In [1131]:
len(splitter(test2))

AttributeError: 'NoneType' object has no attribute 'group'